In [48]:
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.players import BasePokerPlayer
from fish_player import FishPlayer
from pypokerengine.utils.card_utils import estimate_hole_card_win_rate, gen_cards
from pypokerengine.api.emulator import Emulator

In [47]:
hole_card = gen_cards(['H4', 'D7'])
community_card = gen_cards(['D3', 'C5', 'C6'])
estimate_hole_card_win_rate(nb_simulation=1000, nb_player=3, hole_card=hole_card, community_card=community_card)

0.837

In [60]:
class PokerBot(BasePokerPlayer):
    def __init__(self):
        super().__init__()

    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        if self.strong_hand(valid_actions, hole_card, round_state):
            action = "raise"
            amount = valid_actions[2]["amount"]["min"]
        else:
            action = "call"
            amount = valid_actions[1]["amount"]
        return action, amount
    
    def strong_hand(self, valid_actions, hole_card, round_state):
        win_rate = estimate_hole_card_win_rate(
        nb_simulation=1000,
        nb_player=2,
        hole_card=gen_cards(hole_card),
        community_card=gen_cards(round_state['community_card'])
    )
        if win_rate >= 0.6:
            return True
        else:
            return False
    

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [45]:
config = setup_config(max_round=10, initial_stack=1000, small_blind_amount=10)
config.register_player(name="AI_Bot", algorithm=PokerBot())
config.register_player(name="p2", algorithm=FishPlayer())

In [ ]:
game_result = start_poker(config, verbose=1)
print(game_result)

In [51]:
emulator = Emulator()
emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)

In [52]:
class OneActionModel(BasePokerPlayer):

    FOLD, CALL, MIN_RAISE, MAX_RAISE = 0, 1, 2, 3

    def set_action(self, action):
        self.action = action

    def declare_action(self, valid_actions, hole_card, round_state):
        if self.FOLD == self.action:
            return valid_actions[0]['action'], valid_actions[0]['amount']
        elif self.CALL == self.action:
            return valid_actions[1]['action'], valid_actions[1]['amount']
        elif self.MIN_RAISE == self.action:
            return valid_actions[2]['action'], valid_actions[2]['amount']['min']
        elif self.MAX_RAISE == self.action:
            return valid_actions[2]['action'], valid_actions[2]['amount']['max']
        else:
            raise Exception("Invalid action [ %s ] is set" % self.action)

In [59]:
import uuid

# Generate UUIDs for each player
player1_uuid = str(uuid.uuid4())
player2_uuid = str(uuid.uuid4())

# Create instances of the player models
player1_model = OneActionModel()
player1_model.set_action(OneActionModel.CALL)  # For example, always CALL

player2_model = OneActionModel()
player2_model.set_action(OneActionModel.MIN_RAISE)  # For example, always MIN_RAISE

# Register players with the emulator
emulator.register_player(player1_uuid, player1_model)
emulator.register_player(player2_uuid, player2_model)


In [58]:
from pypokerengine.utils.game_state_utils import (
    restore_game_state, attach_hole_card, attach_hole_card_from_deck
)

def setup_game_state(round_state, my_uuid, my_hole_card):
    # Restore the game state from the round state
    game_state = restore_game_state(round_state)
    
    # Assign hole cards to each player
    for player_info in round_state['seats']:
        uuid = player_info['uuid']
        if uuid == my_uuid:
            # We know our own hole cards
            game_state = attach_hole_card(game_state, uuid, my_hole_card)
        else:
            # Assign random hole cards to opponents
            game_state = attach_hole_card_from_deck(game_state, uuid)
    
    return game_state
